This is a Python code to remove attenuate all voxel intensities of the CTA and CEMRA that do not intersect. If a CTA voxel equals -1, set the corresponding CEMRA voxel to -1. If a CEMRA voxel equals the background peak, set the corresponding CTA voxel to -1.

In [ ]:
import os
import nibabel as nib
import numpy as np

def process_pair(cta_path, cemra_path):
    """
    Process a single CTA/CEMRA pair:
      1. Compute the mode (most frequent value) of the CEMRA image, assumed to be the background peak.
      2. For all voxels:
         - If a CTA voxel equals -1, set the corresponding CEMRA voxel to -1.
         - If a CEMRA voxel equals the background peak, set the corresponding CTA voxel to -1.
      3. Save the modified images with a '_processed' suffix.
    """
    # Load the images
    cta_img = nib.load(cta_path)
    cemra_img = nib.load(cemra_path)
    cta_data = cta_img.get_fdata()
    cemra_data = cemra_img.get_fdata()

    # Compute the histogram for CEMRA: use np.unique to get the mode (background peak)
    unique_vals, counts = np.unique(cemra_data, return_counts=True)
    background_peak = unique_vals[np.argmax(counts)]
    print(f"Background peak for {os.path.basename(cemra_path)}: {background_peak}")

    # Create masks for the conditions (vectorized operations)
    mask_cta_minus1 = (cta_data == -1)
    mask_cemra_background = (cemra_data == background_peak)

    # Apply modifications:
    # 1. For all voxels where CTA == -1, force CEMRA voxel to -1
    cemra_data[mask_cta_minus1] = -1

    # 2. For all voxels where CEMRA equals the background peak, force CTA voxel to -1
    cta_data[mask_cemra_background] = -1

    # Create new NIfTI images using the original affine and header information
    cta_new = nib.Nifti1Image(cta_data, affine=cta_img.affine, header=cta_img.header)
    cemra_new = nib.Nifti1Image(cemra_data, affine=cemra_img.affine, header=cemra_img.header)

    # Create output filenames by adding a suffix before the .nii extension
    cta_out_path = cta_path.replace('.nii', '_processed.nii')
    cemra_out_path = cemra_path.replace('.nii', '_processed.nii')

    # Save the processed images
    nib.save(cta_new, cta_out_path)
    nib.save(cemra_new, cemra_out_path)

    print(f"Processed CTA: {cta_path} -> {cta_out_path}")
    print(f"Processed CEMRA: {cemra_path} -> {cemra_out_path}\n")


def process_all_pairs(data_dir):
    """
    Iterate over all CTA image files in data_dir, derive their corresponding CEMRA files,
    and process each pair if the CEMRA file exists.
    """
    # Example: Looking for files ending with 'standardized.nii' (modify as needed)
    cta_files = sorted([f for f in os.listdir(data_dir) if f.endswith("standardized.nii")])
    if not cta_files:
        print("No CTA files found in the specified directory.")
        return

    for cta_file in cta_files:
        cta_path = os.path.join(data_dir, cta_file)
        # Derive corresponding CEMRA path using string replacement.
        cemra_path = cta_path.replace("corrected_", "")
        cemra_path = cemra_path.replace("_resampled_cropped_CTI_standardized.nii", "_registered.nii")
        if os.path.exists(cemra_path):
            process_pair(cta_path, cemra_path)
        else:
            print(f"CEMRA file not found for CTA file: {cta_path}\n")


if __name__ == "__main__":
    # Change this path to the directory containing your CTA and CEMRA image files.
    data_directory = r"C:\Users\willd\Documents\Class_projects\Deep_learning\norm_data\training"
    process_all_pairs(data_directory)


This version removes all blank slices from CTA and CEMRA

In [ ]:
import os
import nibabel as nib
import numpy as np

def process_pair(cta_path, cemra_path):
    """
    Process a single CTA/CEMRA pair:
      1. Compute the histogram (mode) of the original CEMRA image, assumed to be the background peak.
      2. For all voxels:
         - If a CTA voxel equals -1, set the corresponding CEMRA voxel to -1.
         - If a CEMRA voxel equals the background peak, set the corresponding CTA voxel to -1.
      3. After modifications, remove any sagittal slices from the CTA volume (and corresponding CEMRA slices)
         that are entirely -1.
      4. Save the modified images with a '_processed' suffix.
    """
    # Load the images
    cta_img = nib.load(cta_path)
    cemra_img = nib.load(cemra_path)
    cta_data = cta_img.get_fdata()
    cemra_data = cemra_img.get_fdata()

    # Compute the histogram for CEMRA using the original data to get the background peak
    unique_vals, counts = np.unique(cemra_data, return_counts=True)
    background_peak = unique_vals[np.argmax(counts)]
    print(f"Background peak for {os.path.basename(cemra_path)}: {background_peak}")

    # Create masks for the conditions
    mask_cta_minus1 = (cta_data == -1)
    mask_cemra_background = (cemra_data == background_peak)

    # Apply modifications:
    # 1. For all voxels where CTA == -1, force the corresponding CEMRA voxel to -1
    cemra_data[mask_cta_minus1] = -1
    # 2. For all voxels where CEMRA equals the background peak, force the corresponding CTA voxel to -1
    cta_data[mask_cemra_background] = -1

    # Now remove sagittal slices where the modified CTA slice is entirely -1.
    valid_slices = np.any(cta_data != -1, axis=(1, 2))
    cta_data = cta_data[valid_slices, :, :]
    cemra_data = cemra_data[valid_slices, :, :]

    # If all slices are removed, skip processing this volume pair.
    if cta_data.shape[0] == 0:
        print(f"All slices removed for volume {os.path.basename(cta_path)}; skipping.")
        return

    # Create new NIfTI images using the original affine and header information
    cta_new = nib.Nifti1Image(cta_data, affine=cta_img.affine, header=cta_img.header)
    cemra_new = nib.Nifti1Image(cemra_data, affine=cemra_img.affine, header=cemra_img.header)

    # Create output filenames by adding a suffix before the .nii extension
    cta_out_path = cta_path.replace('.nii', '_processed.nii')
    cemra_out_path = cemra_path.replace('.nii', '_processed.nii')

    # Save the processed images
    nib.save(cta_new, cta_out_path)
    nib.save(cemra_new, cemra_out_path)

    print(f"Processed CTA: {cta_path} -> {cta_out_path}")
    print(f"Processed CEMRA: {cemra_path} -> {cemra_out_path}\n")


def process_all_pairs(data_dir):
    """
    Iterate over all CTA image files in data_dir, derive their corresponding CEMRA files,
    and process each pair if the CEMRA file exists.
    """
    # Looking for files ending with 'standardized.nii'
    cta_files = sorted([f for f in os.listdir(data_dir) if f.endswith("standardized.nii")])
    if not cta_files:
        print("No CTA files found in the specified directory.")
        return

    for cta_file in cta_files:
        cta_path = os.path.join(data_dir, cta_file)
        # Derive corresponding CEMRA path using string replacement.
        cemra_path = cta_path.replace("corrected_", "")
        cemra_path = cemra_path.replace("_resampled_cropped_CTI_standardized.nii", "_registered.nii")
        if os.path.exists(cemra_path):
            process_pair(cta_path, cemra_path)
        else:
            print(f"CEMRA file not found for CTA file: {cta_path}\n")


if __name__ == "__main__":
    # Change this path to the directory containing your CTA and CEMRA image files.
    data_directory = r"C:\Users\willd\Documents\Class_projects\Deep_learning\norm_data\testing"
    process_all_pairs(data_directory)


After transferring the intersection files to another directory, remove the "processed" suffix. 

In [ ]:
import os
import argparse

def remove_processed_suffix(directory):
    """
    Iterate over all files in the given directory and remove the '_processed'
    substring from the filename if it exists.
    """
    # List all files in the directory
    for filename in os.listdir(directory):
        if '_processed' in filename:
            # Create new filename by removing the '_processed' substring
            new_filename = filename.replace('_processed', '')
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} -> {new_path}")


remove_processed_suffix(r"C:\Users\willd\Documents\Class_projects\Deep_learning\no_blank_intersection_norm_data\testing")
